#  This is a Simple Student Network

In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.slim as slim
from tensorflow.examples.tutorials.mnist import input_data
import os
import matplotlib.pyplot as plt
# display plots in this notebook
%matplotlib inline

# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

# set gpu device
#os.environ['CUDA_VISIBLE_DEVICES']='0' 

##  1. prepare the train and test data 

In [2]:
#   prepare the data
data="D:/Deep_Learning/TensorFlow/CS 20SI_youtube_video/class_notes/MNIST_data/"
mnist = input_data.read_data_sets(data, one_hot=True)
print("train data :",mnist.train.images.shape)
print("test  data :",mnist.test.images.shape)

Extracting D:/Deep_Learning/TensorFlow/CS 20SI_youtube_video/class_notes/MNIST_data/train-images-idx3-ubyte.gz
Extracting D:/Deep_Learning/TensorFlow/CS 20SI_youtube_video/class_notes/MNIST_data/train-labels-idx1-ubyte.gz
Extracting D:/Deep_Learning/TensorFlow/CS 20SI_youtube_video/class_notes/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting D:/Deep_Learning/TensorFlow/CS 20SI_youtube_video/class_notes/MNIST_data/t10k-labels-idx1-ubyte.gz
train data : (55000, 784)
test  data : (10000, 784)


##  2.  Define student Network

In [3]:
x = tf.placeholder(tf.float32,[None, 784], name='Data')
y_ = tf.placeholder(tf.float32, [None, 10], name='s_label')

W1 = tf.Variable(tf.truncated_normal([784, 800]),name='s_W1')
b1 = tf.Variable(tf.truncated_normal([800]),name='s_b1')
h_1 = tf.matmul(x, W1,name='h_1')
h_1 = tf.nn.relu(tf.add(h_1, b1,name='h_1_b'))

W2 = tf.Variable(tf.truncated_normal([800,800]),name='s_W2')
b2 = tf.Variable(tf.truncated_normal([800]),name='s_b2')
h_2 = tf.matmul(h_1, W2, name='h_2')
h_2 = tf.nn.relu(tf.add(h_2, b2 , name='h_2_b'))

W3 = tf.Variable(tf.truncated_normal([800,10]), name='s_W3')
b3 = tf.Variable(tf.truncated_normal([10]), name='s_b3')
logits = tf.matmul(h_2, W3, name='s_logits')
logits = tf.add(logits, b3, name='s_logits_b')

##  3. Define Loss and Optimiator

In [4]:
# loss
logits_pre = logits - tf.reduce_max(logits)
loss_pre = tf.nn.softmax_cross_entropy_with_logits(logits=logits_pre,labels=y_, name='hd_loss_pre')
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_pre,labels=y_, name='hd_loss'))

# Define Optimizer
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

correct_prediction =tf.equal(tf.argmax(logits,1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),name='accurarcy_s')

# Define Log Summary
tf.summary.scalar('loss', loss)
tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge_all()
saver = tf.train.Saver()

## 4. Define Session and train  processing

In [6]:
tf.device("/cpu:0")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter("./log",sess.graph)
    for i in range(11000):
        batch = mnist.train.next_batch(100)
        
        if i%200 == 0:
            summary = sess.run(merged, feed_dict={x:mnist.validation.images, y_:mnist.validation.labels})
            loss_, acc_ = sess.run([loss, accuracy], feed_dict={x:mnist.validation.images, y_:mnist.validation.labels})
            writer.add_summary(summary,i)
            #print("iter %d"% i)
            print("iter %d  loss : %g,  accuracy = %g" %  (i, loss_, acc_))
            
        train_step.run(feed_dict={x:batch[0], y_:batch[1]})
        
    print("Final test accuracy : %g" % sess.run(accuracy, feed_dict={x:mnist.test.images, y_ : mnist.test.labels}))
    print("train Done.")
    #"""
    

iter 0  loss : 5362.22,  accuracy = 0.0854
iter 200  loss : 144.812,  accuracy = 0.8558
iter 400  loss : 91.5502,  accuracy = 0.904
iter 600  loss : 71.2664,  accuracy = 0.9182
iter 800  loss : 62.1026,  accuracy = 0.926
iter 1000  loss : 53.2908,  accuracy = 0.9312
iter 1200  loss : 48.4591,  accuracy = 0.9324
iter 1400  loss : 45.3006,  accuracy = 0.9404
iter 1600  loss : 40.9602,  accuracy = 0.9456
iter 1800  loss : 43.2199,  accuracy = 0.938
iter 2000  loss : 38.8856,  accuracy = 0.9436
iter 2200  loss : 35.2552,  accuracy = 0.9484
iter 2400  loss : 35.4633,  accuracy = 0.9522
iter 2600  loss : 36.9727,  accuracy = 0.9512
iter 2800  loss : 32.5431,  accuracy = 0.9534
iter 3000  loss : 34.9655,  accuracy = 0.952
iter 3200  loss : 32.6741,  accuracy = 0.9516
iter 3400  loss : 31.2108,  accuracy = 0.9548
iter 3600  loss : 30.6269,  accuracy = 0.9554
iter 3800  loss : 34.3033,  accuracy = 0.9506
iter 4000  loss : 30.6043,  accuracy = 0.9564
iter 4200  loss : 32.587,  accuracy = 0.9566
